# Model Training

## Model

In [14]:
import os
import random
import shutil
import string
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

# Define constants
input_shape = (256, 256, 1)  # Image dimensions and grayscale
original_dataset_dir = r"C:\Users\ap2935\Desktop\asl_dataset\PRO"  
split_dataset_dir = r"C:\Users\ap2935\Desktop\asl_dataset"  
train_dir = os.path.join(split_dataset_dir, "train")
validation_dir = os.path.join(split_dataset_dir, "validation")
test_dir = os.path.join(split_dataset_dir, "test")

# Function to split the dataset
def split_dataset(original_dir, split_dir, train_ratio=0.7, validation_ratio=0.2, test_ratio=0.1):
    if not os.path.exists(split_dir):
        os.makedirs(split_dir)
    
    for subset in ["train", "validation", "test"]:
        os.makedirs(os.path.join(split_dir, subset), exist_ok=True)
    
    for class_name in os.listdir(original_dir):
        class_dir = os.path.join(original_dir, class_name)
        if not os.path.isdir(class_dir):
            continue

        files = os.listdir(class_dir)
        random.shuffle(files)
        total_files = len(files)

        # Split indices
        train_end = int(total_files * train_ratio)
        validation_end = train_end + int(total_files * validation_ratio)

        # Create splits
        train_files = files[:train_end]
        validation_files = files[train_end:validation_end]
        test_files = files[validation_end:]

        # Copy files to respective directories
        for file in train_files:
            os.makedirs(os.path.join(split_dir, "train", class_name), exist_ok=True)
            shutil.copy(os.path.join(class_dir, file), os.path.join(split_dir, "train", class_name, file))
        for file in validation_files:
            os.makedirs(os.path.join(split_dir, "validation", class_name), exist_ok=True)
            shutil.copy(os.path.join(class_dir, file), os.path.join(split_dir, "validation", class_name, file))
        for file in test_files:
            os.makedirs(os.path.join(split_dir, "test", class_name), exist_ok=True)
            shutil.copy(os.path.join(class_dir, file), os.path.join(split_dir, "test", class_name, file))

# Split the dataset
split_dataset(original_dataset_dir, split_dataset_dir)

# Function to apply data augmentation
def apply_data_augmentation(train_dir):
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    return train_datagen.flow_from_directory(
        train_dir,
        target_size=(input_shape[0], input_shape[1]),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical'
    )

# Validation generator
def create_validation_generator(validation_dir):
    validation_datagen = ImageDataGenerator(rescale=1./255)
    return validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(input_shape[0], input_shape[1]),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical'
    )

# Test generator
def create_test_generator(test_dir):
    test_datagen = ImageDataGenerator(rescale=1./255)
    return test_datagen.flow_from_directory(
        test_dir,
        target_size=(input_shape[0], input_shape[1]),
        batch_size=32,
        color_mode='grayscale',
        class_mode='categorical'
    )

# Build a more complex CNN model
def build_cnn_model(num_classes):
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))  # Regularization

    # Block 2
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.25))

    # Block 3
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(128, (3, 3), activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Dropout(0.5))

    # Global Average Pooling
    model.add(layers.GlobalAveragePooling2D())

    # Fully Connected Layer
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))

    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

# Function to compile the model
def compile_model(model):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Function to train the model
def train_model(model, train_generator, validation_generator, epochs=10):
    model.fit(train_generator, validation_data=validation_generator, epochs=epochs)

# Function to evaluate the model
def evaluate_model(model, test_generator):
    return model.evaluate(test_generator)

# Apply data augmentation
train_generator = apply_data_augmentation(train_dir)
validation_generator = create_validation_generator(validation_dir)
test_generator = create_test_generator(test_dir)

# Build the CNN model
cnn_model = build_cnn_model(num_classes=len(train_generator.class_indices))

# Compile the model
compile_model(cnn_model)



Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


C:\Users\ap2935\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# Train the model
train_model(cnn_model, train_generator, validation_generator, epochs=10)

# Evaluate the model
test_loss, test_accuracy = evaluate_model(cnn_model, test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


C:\Users\ap2935\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


ValueError: Must provide at least one structure

In [ ]:
# Evaluate the model on the test set
evaluation_result = evaluate_model(cnn_model, test_generator)
print("Evaluation Result:", evaluation_result)

# Make predictions on the test set
predictions = cnn_model.predict(test_generator)

# Get the class labels
class_labels = list(test_generator.class_indices.keys())

# Loop through each prediction and print the class label with its probability
for i, prediction in enumerate(predictions):
    predicted_class_index = tf.argmax(prediction).numpy()
    predicted_class_label = class_labels[predicted_class_index]
    predicted_probability = prediction[predicted_class_index]

    print(f"Sample {i + 1}: Predicted Class - {predicted_class_label}, Probability - {predicted_probability:.4f}")

# Save the model to a specified path
save_path = '/Users/aditya/Documents/GitHub/ASL/MODEL17' 
cnn_model.save(save_path)
print(f"Model saved to {save_path}")


## Prediction Functions

In [9]:
import cv2
import rembg
from PIL import Image
import numpy as np
import random
from IPython.display import Image as IPImage, display

# Function to capture an image from the default camera, remove the background, and preprocess it
def capture_and_preprocess():
    # Open the default camera (camera index 0)
    cap = cv2.VideoCapture(0)

    # Allow the camera to adjust (you may need to adjust the delay based on your camera)
    cv2.waitKey(1000)

    # Read a frame from the camera
    ret, frame = cap.read()

    # Release the camera capture
    cap.release()

    if ret:
        # Convert the OpenCV frame to a PIL Image
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Use rembg to remove the background
        with rembg.remove(pil_image) as result:
            # Convert the result to an OpenCV format
            sign = cv2.cvtColor(np.array(result), cv2.COLOR_RGBA2BGRA)

        # Convert the image to grayscale
        grayscale_image = cv2.cvtColor(sign, cv2.COLOR_BGRA2GRAY)

        # Resize the image to the target size (256x256)
        target_size = (256, 256)
        resized_image = cv2.resize(grayscale_image, target_size)

        # Remove random noise (salt and pepper noise)
        noisy_image = Image.fromarray(resized_image)
        noisy_image = noisy_image.point(lambda p: p + random.choice([-50, 0, 50]) if random.random() < 0.05 else p)

        # Convert the noisy image back to NumPy array
        noisy_image_np = np.array(noisy_image)

        return frame, noisy_image_np
    else:
        print("Failed to capture an image.")
        return None

from gtts import gTTS
from IPython.display import Audio
from io import BytesIO

def text_to_speech(text):
    # Create a gTTS object with the desired text
    tts = gTTS(text=text, lang='en')

    # Save the generated speech to a BytesIO object
    audio_stream = BytesIO()
    tts.write_to_fp(audio_stream)
    audio_stream.seek(0)

    # Display the generated audio
    return Audio(data=audio_stream.read(), autoplay=True)





# Prediction

In [10]:
import cv2
import rembg
from PIL import Image
import numpy as np
import random

# Function to capture an image from the default camera, remove the background, and preprocess it
def capture_and_preprocess():
    # Open the default camera (camera index 0)
    cap = cv2.VideoCapture(0)

    # Allow the camera to adjust (you may need to adjust the delay based on your camera)
    cv2.waitKey(1000)

    # Read a frame from the camera
    ret, frame = cap.read()

    # Release the camera capture
    cap.release()

    if ret:
        # Convert the OpenCV frame to a PIL Image
        pil_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Use rembg to remove the background
        result = rembg.remove(pil_image)
        sign = cv2.cvtColor(np.array(result), cv2.COLOR_RGBA2GRAY)

        # Resize the image to the target size (256x256)
        target_size = (256, 256)
        resized_image = cv2.resize(sign, target_size)

        # Denoise the image (optional)
        denoised_image = cv2.fastNlMeansDenoising(resized_image, None, 30, 7, 21)

        return frame, denoised_image
    else:
        print("Failed to capture an image.")
        return None, None


c:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [11]:
# Capture an image, remove the background, and preprocess it
input_image, preprocessed_image = capture_and_preprocess()

# Check if the preprocessed image was successfully captured
if preprocessed_image is not None:
    # Display the input image
    display(IPImage(data=cv2.imencode('.png', input_image)[1].tobytes(), format='png'))

    # Display the preprocessed image
    display(IPImage(data=cv2.imencode('.png', preprocessed_image)[1].tobytes(), format='png'))

    print("Image captured, background removed, and preprocessed.")
else:
    print("Image capture failed.")


# Assuming 'preprocessed_image' is your preprocessed image
input_array = np.expand_dims(preprocessed_image, axis=0)

# Make predictions on the input image
predictions = cnn_model.predict(input_array)



# Get the class labels
class_labels = list(train_generator.class_indices.keys())

# Find the class label with the highest probability
predicted_class_index = np.argmax(predictions)
predicted_class_label = class_labels[predicted_class_index]
predicted_probability = predictions[0, predicted_class_index]

# Print the predicted class label and its probability
print(f"Predicted Class - {predicted_class_label}, Probability - {predicted_probability:.4f}")


print(predicted_class_label)

text_input = predicted_class_label
audio_output = text_to_speech(text_input)

audio_output

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1367: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvWaitKey'
